In [2]:
import sys
import os
import gzip
import shutil
import json
import pandas as pd
import pickle
import numpy as np
%matplotlib inline

## Preprocess treatment data

In [4]:
### Interpreting json file
df = pickle.load(open('data/entries_20396154_2.pkl', 'rb'))

In [7]:
len(df['sgv'].values)

32418

In [12]:
df_np = df['sgv'].values[:32400].reshape((1080,30,1))

In [25]:
df_np[0]

array([[124],
       [118],
       [114],
       [111],
       [109],
       [107],
       [105],
       [104],
       [106],
       [106],
       [105],
       [104],
       [102],
       [ 98],
       [ 93],
       [ 88],
       [ 85],
       [ 83],
       [ 81],
       [ 81],
       [ 83],
       [ 84],
       [ 85],
       [ 86],
       [ 92],
       [100],
       [112],
       [123],
       [133],
       [140]])

In [16]:
np.save('/home/jupyter/work/demo/tensor_train_RNN/entries.npy', df_np)

In [28]:
data_test2 = np.load('/home/jupyter/work/demo/tensor_train_RNN/entries.npy')

In [21]:
data_test = np.load('/home/jupyter/work/demo/tensor_train_RNN/data.npy')

In [27]:
data_test.shape

(1000, 100, 1)

In [29]:
data_test2.shape

(1080, 30, 1)

In [18]:
np.save('/home/jupyter/work/demo/tensor_train_RNN/entries_60.npy', df['sgv'].values[:32400].reshape((540,60,1)))

In [3]:
def transfer_dataframe(patient_id, file_name):
    with open(file_name) as file:
        j_object = json.load(file)
        df = pd.DataFrame(j_object)
        try:
            df['datetime'] = pd.to_datetime(df['timestamp'], utc=True)
        except:
            print(df['timestamp'].head())
            return None
            
        df['pid'] = patient_id
        df.sort_values(by='datetime', inplace=True)
        df_result = df[['pid', 'datetime', 'eventType', 'insulin','carbs','absolute','rate','duration']]

    return df_result

In [5]:
## these samples have less data missing problems
samples = ['96254963', '57176789', '20396154', '00897741',
       '66836068', '66019205', '95851255', '97417885', '20216809',
       '47750728', '60844515', '37764532', '43589707', '14092221',
       '96805916', '95614431', '21946407', '37948668', '25692073',
       '99848889', '15634563', '80796147']  

In [ ]:
## merge treatment files person to person

count = 0
path = '/Users/wang/data/OpenAPS'
for folder in samples:
    print('PID:', folder)
    record_path = path + '/' +folder+'/direct-sharing-31'
    if os.path.exists(record_path):
        files = os.listdir(record_path)
        df_folder = pd.DataFrame([], columns=['pid', 'datetime', 'eventType', 'insulin','carbs','absolute','rate','duration'])
        for file in files:
            if file.find('treatments') > -1 and file.endswith('.json'):
                print('file', file)
                df = transfer_dataframe(folder, record_path+'/'+file)
                if df is None:
                    continue
                df_folder = df_folder.append(df)
#         print('before drop', len(df_folder))
#         df_folder.drop_duplicates('datetime',inplace = True)
#         print('after drop', len(df_folder))
#         print('saving result...')
        pickle.dump(df_folder, open(path + '/treat_dfs/'+folder+'_treats.pkl', 'wb'))
# z


In [ ]:
# Synthesize treatment with entries

path_treat = '/Users/wang/data/OpenAPS/treat_dfs'
path_entries = '/Users/wang/data/OpenAPS/entry_dfs'
start_date = '2018-01-01'
end_date = '2018-01-31'

dfs = []
for sample in samples:
    #treats
    df_t = pickle.load(open(path_treat+'/'+sample+'_treats.pkl', 'rb'))
    df_t.set_index('datetime', inplace=True)
    df_t = df_t.tz_convert(None)
    df_t_2018 = df_t[start_date:end_date]
    #bg entries
    df_e = pickle.load(open(path_entries+'/'+sample+'_entries.pkl', 'rb'))
    df_e.set_index('datetime', inplace=True)
    df_e = df_e.tz_convert(None)
    df_e_2018 = df_e[start_date:end_date]
    
    df_2018 = pd.merge(df_e_2018,df_t_2018, on = 'pid', how='left' )
    df_2018.sort_index(inplace = True)
    print(df_2018.head(3))
    dfs.append(df_2018)
    
dfs_all = pd.concat(dfs)
pickle.dump(df_folder, open('/Users/wang/data/OpenAPS/sample_entry_treats.pkl', 'wb'))


        pid    sgv eventType insulin carbs absolute rate duration
0  27700103  134.0       NaN     NaN   NaN      NaN  NaN      NaN
1  27700103  138.0       NaN     NaN   NaN      NaN  NaN      NaN
2  27700103  148.0       NaN     NaN   NaN      NaN  NaN      NaN
        pid    sgv   eventType  insulin  carbs  absolute rate  duration
0  96254963  165.0  Temp Basal      NaN    NaN       0.0    0      30.0
1  96254963  165.0  Temp Basal      NaN    NaN       0.0    0      60.0
2  96254963  165.0  Temp Basal      NaN    NaN       0.0    0      30.0
        pid  sgv   eventType  insulin  carbs  absolute rate  duration
0  57176789  118  Temp Basal      NaN    NaN       0.8  0.8      30.0
1  57176789  118  Temp Basal      NaN    NaN       1.2  1.2      30.0
2  57176789  118  Temp Basal      NaN    NaN       1.9  1.9      30.0
        pid  sgv   eventType  insulin  carbs  absolute  rate  duration
0  20396154  220  Temp Basal      NaN    NaN      0.00     0       0.0
1  20396154  220  Temp Bas

In [22]:
dfs_all['duration'].describe()

count    1.645022e+07
mean     3.332370e+01
std      4.195641e+01
min      0.000000e+00
25%      3.000000e+01
50%      3.000000e+01
75%      3.000000e+01
max      7.200000e+02
Name: duration, dtype: float64

In [45]:
treats_df.columns

Index(['absolute', 'carbs', 'eventType', 'glucose', 'insulin', 'notes', 'pid',
       'rate', 'ratio'],
      dtype='object')

In [23]:
treats_df['eventType'].unique()

NameError: name 'treats_df' is not defined

In [44]:
entries_df.describe()

,pid,sgv
count,336123,334934.0
unique,41,373.0
top,74077367,110.0
freq,16951,3628.0


In [ ]:
df_2018_1 = pd.merge(treats_df, entries_df, on='pid')